In [ ]:
!pip install -Uqq fastbook 
import fastbook 
fastbook.setup_book()

In [ ]:
from fastai.vision.all import * 
from fastbook import * 

[Further Research]()

1. Create your own implementation of `Learner` from scratch, based on the 

In [ ]:
from fastai.vision.all import *
from fastbook import * 


def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets == 1, 1 - predictions, predictions).mean()


def build_dset(path_three, path_seven):
    threes = _build_tensor_from_image(path_three)
    sevens = _build_tensor_from_image(path_seven)
    return _build_dset(threes, sevens)


def _build_dset(threes, sevens):
    x = torch.cat([threes, sevens])
    x = x.view(-1, 28 * 28)
    y = tensor([1] * len(threes) + [0] * len(sevens))
    y = y.unsqueeze(1)
    return list(zip(x, y))
    
    
def _build_tensor_from_image(path):
    x = path.ls().sorted()
    x = [tensor(Image.open(image_path)) for image_path in x]
    return torch.stack(x).float() / 255


def batch_accuracy(xb, yb):
    predictions = xb.sigmoid()
    correct = (predictions > 0.5) == yb
    return correct.float().mean()

    
class BasicOptim:
    def __init__(self, params, learning_rate):
        self.params = list(params)
        self.learning_rate = learning_rate
        
    def step(self, *args, **kwargs):
        for param in self.params:
            param.data -= param.grad.data * self.learning_rate
            
    def zero_grad(self, *args, **kwargs):
        for param in self.params:
            param.grad = None 
            
            
class CustomLearner:
    def __init__(self, dls, model, opt, loss_func, metrics=accuracy):
        self.train_dl = dls[0]
        self.valid_dl = dls[1]
        self.model = model 
        self.opt = opt
        self.loss_func = loss_func
        self.metrics = metrics
        
    def fit(self, n_epochs, learning_rate):
        self.train_model(n_epochs)
        
    def train_model(self, n_epochs):
        for _ in range(n_epochs):
            self.train_epoch()
            print(self.validate_epoch(), end="\n")
            
    def train_epoch(self):
        for xb, yb in self.train_dl:
            self.calc_grad(xb, yb)
            self.opt.step()
            self.opt.zero_grad()
            
    def calc_grad(self, xb, yb):
        predictions = self.model(xb)
        loss = self.loss_func(predictions, yb)
        loss.backward()
            
    def validate_epoch(self):
        accuracies = [self.metrics(self.model(xb), yb) for xb, yb in self.valid_dl]
        return round(torch.stack(accuracies).mean().item(), 4)

path = untar_data(URLs.MNIST_SAMPLE)
path.ls()

train_dset = build_dset(path/"train"/"3", path/"train"/"7")
valid_dset = build_dset(path/"valid"/"3", path/"valid"/"7")

train_dl = DataLoader(train_dset, batch_size=256)
valid_dl = DataLoader(valid_dset, batch_size=256)

dls = DataLoaders(train_dl, valid_dl)

linear_model = nn.Linear(28 * 28, 1)

opt = BasicOptim(linear_model.parameters(), 1.)

custom_learner = CustomLearner(
    dls, 
    linear_model, 
    opt=opt,
    loss_func=mnist_loss,
    metrics=batch_accuracy,
)

custom_learner.fit(5, 0.1)

2. Complete all the steps in this chapter using the full MNIST datasets (that is, for all digits, not just 3s and 7s).

In [ ]:
from fastai.vision.all import *
from fastbook import *

def build_x(path):
    x = get_image_files(path).sorted()
    x = [tensor(Image.open(image_path)) for image_path in x]
    x = torch.stack(x)
    x = x.float() / 255
    return x.view(-1, 28 * 28)

def build_y(path):
    y = get_image_files(path).sorted()
    y = [int(image_path.parent.name) for image_path in y]
    return tensor(y)

def build_dl(x, y):
    dset = list(zip(x, y))
    return DataLoader(dset, batch_size=256, shuffle=True)

path = untar_data(URLs.MNIST)
path.ls()


(train_x, train_y) = (build_x(path/"training"), build_y(path/"training"))
train_dl = build_dl(train_x, train_y)

(valid_x, valid_y) = (build_x(path/"testing"), build_y(path/"testing"))
valid_dl = build_dl(valid_x, valid_y)

dls = DataLoaders(train_dl, valid_dl)

simple_net = nn.Sequential(
    nn.Linear(28 * 28, 50),
    nn.ReLU(),
    nn.Linear(50, 10)
)

learner = Learner(
    dls, 
    simple_net, 
    opt_func=SGD, 
    loss_func=nn.CrossEntropyLoss(),
    metrics=accuracy,
    lr=0.001
)

learner.fit(10, 0.1)